# Do Zero ao Som: Criando Áudio Digital com Python

In [57]:
# Antes de tudo, instale as dependências

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import ipywidgets as widgets
from ipywidgets import interactive
from scipy.io import wavfile
from scipy import signal
from abc import ABC, abstractmethod

## Introdução

> *Como o computador processa uma música digitalmente?*
>
> *Como podemos entender uma representação digital de um sinal de áudio?*
>
> *Como sintetizadores digitais são desenvolvidos?*

Neste workshop, vamos entender um pouco sobre o que é um sinal de áudio digital e como podemos criar sons a partir do zero utilizando Python. Tudo isso de maneira interativa e prática.

Divirta-se!

## Som e Sinal

Vamos ganhar uma intuição intuição inicial sobre o que é som e o que é sinal sonoro. Essas definições são importantes para entendermos o que estamos fazendo quando mexemos com áudio digital.

### O que é som?

**Som** é um fenômeno físico que ocorre quando moléculas de ar (ou outro meio elástico) se comprimem e rarefazem rapidamente.
Alto-falantes, por exemplo, movem-se para frente e para trás rapidamente, comprimindo e rarefazendo o ar, criando som.
Por sua vez, nossos ouvidos são capazes de captar essas variações de pressão do ar e transformá-las em sinais elétricos que são enviados ao nosso cérebro.

Podemos modelar esse movimento do ar com a ajuda de computadores e da matemática.

### O que é sinal sonoro?

Um **sinal sonoro** pode ser entendido como uma função matemática que descreve a **variação** de pressão do ar ou tensão elétrica ao longo do tempo.
Podemos visualizar um sinal sonoro com um gráfico onde o eixo horizontal representa o tempo e o eixo vertical representa a amplitude do som em cada instante de tempo.

Caso você nunca tenha visto a onda de uma música antes, observe o sinal dos primeiros 60 segundos da música "*No Ordinary Love*" da cantora Sade no software Ableton Live.

> *This is no ordinary love...*

### Sinais analógicos e digitais

Consegue perceber que temos vários pontos no gráfico da onda sonora? Cada um desses pontos representa a intensidade do som em um instante de tempo.
Por isso, podemos dizer que esse sinal sonoro é **discreto** ou **digital**.

Portanto, podemos imaginar um arquivo de áudio como uma grande lista de números.

In [58]:
sample_rate, audio_file = wavfile.read('Sade-No_Ordinary_Love-60s.wav')

# Mostra o NumPy Dtype
print(audio_file.dtype) # int16, de [-32768, +32767]

# Imprime uma parte do arquivo de áudio
print(audio_file[480000:480000+10])

int16
[[-11313 -14304]
 [-10939 -14153]
 [-11831 -14842]
 [-12245 -15099]
 [-11796 -14719]
 [-11042 -13983]
 [-11059 -13645]
 [-10910 -13119]
 [ -9690 -11869]
 [ -9067 -11190]]


> Dizemos que um sinal sonoro é **analógico** quando a função está definida para todos os instantes no tempo. Ou seja, o sinal sonoro é **contínuo**. Sinais analógicos estão presentes no mundo real, como o som que ouvimos, e também em equipamentos de áudio, como microfones e alto-falantes.
>
> Dizemos que um sinal sonoro é **digital** quando a função está definida apenas para uma lista finita de instantes no tempo. Ou seja, o sinal sonoro é **discreto**. Sinais digitais são utilizados em computadores e equipamentos digitais.

## Amostragem e Quantização

Agora sabemos que um sinal sonoro digital é discretizado no eixo do tempo e também no eixo da amplitude. Mas como isso é feito? Vamos entender um pouco sobre **amostragem** e **quantização**.

> **Amostragem** é o processo de discretização do sinal no eixo do **tempo**.
> Definimos a **taxa de amostragem** como o número de amostras por segundo. A unidade de medida é **Hertz** (Hz). É comum vermos taxas de amostragem de 44.1 kHz, 48 kHz, 96 kHz, etc.

> **Quantização** é o processo de discretização do sinal no eixo da **amplitude**.
> Definimos a **resolução** como o número de níveis de amplitude possíveis. A unidade de medida é **bits**. É comum vermos resoluções de potência de 2, como 8 bits, 16 bits, 24 bits, etc.

Abaixo está um ambiente interativo para você experimentar como uma onda senoidal muda com diferentes frequências, taxas de amostragem e resoluções.

In [59]:
def visualiza_amostragem_e_quantizacao(taxa_amostragem=1000, bits=16, freq=1, mostrar_continuo=False):
    # Parâmetros para visualização
    duracao = 1.0
    
    # Sinal contínuo (com mais pontos para parecer contínuo)
    t_continuo = np.linspace(0, duracao, 10000)
    sinal_continuo = np.sin(2 * np.pi * freq * t_continuo)
    
    # Amostragem
    t_amostrado = np.linspace(0, duracao, int(taxa_amostragem * duracao))
    onda = np.sin(2 * np.pi * freq * t_amostrado)
    
    # Quantização
    resolucao = 2**bits
    onda_quantizada = -1 + 2**(-bits) + 2*np.floor((resolucao-1e-8) * (0.5 + 0.5*onda)) / (resolucao)
    
    # Cria o gráfico
    plt.figure(figsize=(18, 6))
    
    # Mostra o sinal contínuo se o toggle estiver ativado
    if mostrar_continuo:
        plt.plot(t_continuo, sinal_continuo, 'b-', label='Sinal Contínuo', alpha=0.5)
    
    plt.plot(t_amostrado, onda_quantizada, 'r.', label='Sinal Digital')
    
    # Adiciona linhas verticais tracejadas para indicar os pontos de amostragem
    if mostrar_continuo and len(t_amostrado) < 100:  # Limita para não sobrecarregar o gráfico
        for i in range(len(t_amostrado)):
            plt.axvline(x=t_amostrado[i], color='gray', linestyle='--', alpha=0.3)
    
    plt.grid(True)
    plt.xlabel('Tempo (s)')
    plt.ylabel('Amplitude')
    plt.title(f'Senoide ({freq} Hz) - {taxa_amostragem} Hz de amostragem, {bits} bits de resolução')
    plt.legend()
    plt.show()
    
    # Gera sinal completo para reprodução
    # t_completo = np.linspace(0, 1, taxa_amostragem)
    # onda_completa = np.sin(2 * np.pi * freq * t_completo)
    # onda_quantizada_completa = np.round(onda_completa * (resolucao / 2)) / (resolucao / 2)
    
    # Reproduz o áudio
    display(Audio(onda_quantizada, rate=taxa_amostragem))

# Cria a interface interativa
interactive_plot = interactive(
    visualiza_amostragem_e_quantizacao,
    taxa_amostragem=widgets.IntSlider(min=1, max=1000, step=100, value=1000),
    bits=widgets.IntSlider(min=1, max=16, step=1, value=16),
    freq=widgets.IntSlider(min=1, max=440, step=10, value=1),
    mostrar_continuo=widgets.Checkbox(value=False, description='Mostrar sinal contínuo')
)
display(interactive_plot)

interactive(children=(IntSlider(value=1000, description='taxa_amostragem', max=1000, min=1, step=100), IntSlid…

### Frequência de Nyquist

## Sinais elementares

Vamos estudar alguns sinais elementares que são importantes para entendermos o som: **ruídos** e **sinais periódicos**.

### Ruídos

Em computação musical, **ruído** não possui uma definição categórica, mas é comum entendermos ele como um sinal sonoro que:

* Não possui uma altura musical definida
* Não costuma ser periódico

Uma caracterização comum de ruídos é por sua **densidade espectral**, ou seja, a distribuição de energia do sinal em função da frequência. Esses ruídos possuem nomes de cores, como **branco**, **vermelho**, **rosa**, etc.

Vamos sintetizar e ouvir os ruídos branco e vermelho.

**ATENÇÃO: ABAIXE (MUITO) O VOLUME DO SEU COMPUTADOR ANTES DE OUVIR OS ÁUDIOS.**

In [60]:
# Vamos definir uma variável global para a taxa de amostragem
taxa_amostragem = 44100

In [137]:
class Ruido(ABC):
    def __init__(self, amp=1.0, duracao=1.0, freq=None):
        self.amp     = amp
        self.duracao = duracao
        self.freq    = freq
        self.sinal   = self.gera_sinal(amp=self.amp, duracao=self.duracao)
    
    @abstractmethod
    def gera_sinal(self, amp=1.0, duracao=1.0, freq=None):
        pass
    
    def cria_ambiente_interativo(self):
        def ruido_wrapper(amp, duracao):
            # Gera o sinal
            t = np.linspace(0, duracao, int(taxa_amostragem * duracao))
            self.sinal = self.gera_sinal(amp=amp, duracao=duracao, freq=None)
            
            # Gera o gráfico
            plt.figure(figsize=(24, 6))
            plt.plot(t, self.sinal, 'b-')
            plt.grid(True)
            plt.xlabel('Tempo (s)')
            plt.ylabel('Amplitude')
            plt.ylim(-1.1, 1.1)
            plt.title('Sinal de Ruído')
            plt.show()
            
            # Reproduz o áudio
            display(Audio(self.sinal, rate=taxa_amostragem))
        
        # Cria o ambiente interativo
        interactive_plot = interactive(
            ruido_wrapper,
            amp=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=self.amp),
            duracao=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=self.duracao)
        )
        display(interactive_plot) 

#### Ruído branco

O **ruído branco** é um sinal sonoro que possui energia uniformemente distribuída em todas as frequências audíveis.

Talvez você já tenha ouvido ruído branco em um rádio fora de sintonia ou em uma televisão sem sinal. Cuidado, pois esse som pode te incomodar muito!

Vamos sintetizar um ruído branco e ouvi-lo:

In [138]:
class RuidoBranco(Ruido):
    def __init__(self, amp=1.0, duracao=1.0, freq=None):
        super().__init__(amp=amp, duracao=duracao, freq=None)
    
    def gera_sinal(self, amp=1.0, duracao=1.0, freq=None):
        amostras = int(taxa_amostragem * duracao)
        return amp * np.random.uniform(-1, 1, amostras)

In [139]:
ruido_branco = RuidoBranco(amp=0.5, duracao=1.0)
ruido_branco.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=0.5, description='amp', max=1.0, min=0.1), FloatSlider(value=1.0, desc…

#### Ruído vermelho

O **ruído vermelho** é um ruído que possui energia inversamente proporcional à frequência. Isso significa que ele possui mais energia em frequências baixas e menos energia em frequências altas.

O ruído vermelho é um ruído mais agradável de se ouvir do que o ruído branco.

Vamos sintetizar um ruído vermelho e ouvi-lo:

In [140]:
class RuidoVermelho(Ruido):
    def __init__(self, amp=1.0, duracao=1.0, freq=None):
        super().__init__(amp=amp, duracao=duracao, freq=None)
        
    def gera_sinal(self, amp=1.0, duracao=1.0, freq=None):
        amostras = int(taxa_amostragem * duracao)

        branco = np.random.uniform(-1, 1, amostras) # O ruído vermelho é derivado do ruído branco

        vermelho = np.zeros(1 * amostras)
        vermelho[0] = branco[0] # Inicializa a primeira amostra

        for i in range(1, amostras):
            vermelho[i] = branco[i] + vermelho[i - 1]

        vermelho = vermelho / np.max(np.abs(vermelho))
        
        vermelho = amp * vermelho # ajuste de amplitude

        return vermelho

In [141]:
ruido_vermelho = RuidoVermelho(amp=1.0, duracao=1.0)
ruido_vermelho.cria_ambiente_interativo() 

interactive(children=(FloatSlider(value=1.0, description='amp', max=1.0, min=0.1), FloatSlider(value=1.0, desc…

#### *Qual a utilidade de ruídos na síntese sonora?*

Podemos concordar que esses ruídos por si só não têm um aspecto musical (ou têm?). Entretanto, a partir de um ruído, podemos modificá-lo (com filtros, envelopes, efeitos, etc.) para criar sons mais complexos e também mais musicais. Vamos explorar isso mais adiante quando falarmos sobre envelopes ADSR.

### Sinais periódicos

Os sinais periódicos elementares são as **ondas senoidais**, **ondas quadradas**, **ondas dente-de-serra** e **ondas triangulares** , cada um com suas características tonais.

In [142]:
class Onda(ABC):
    def __init__(self, amp=1.0, duracao=1.0, freq=440):
        self.amp     = amp
        self.duracao = duracao
        self.freq    = freq
        self.sinal   = self.gera_sinal(self.amp, self.duracao, self.freq)
    
    @abstractmethod
    def gera_sinal(self, amp=1.0, duracao=1.0, freq=440):
        pass
    
    def cria_ambiente_interativo(self):
        # Cria uma função wrapper para ser chamada pelo widget interativo
        def periodico_wrapper(amp, duracao, freq):
            t = np.linspace(0, duracao, int(taxa_amostragem * duracao))
            self.sinal = self.gera_sinal(amp=amp, duracao=duracao, freq=freq)
            
            # Gera o gráfico
            plt.figure(figsize=(24, 6))
            plt.plot(t, self.sinal, 'b-', label='Onda')
            plt.grid(True)
            plt.xlabel('Tempo (s)')
            plt.ylabel('Amplitude')
            plt.ylim(-1.1, 1.1)
            plt.title(f'Onda - {freq} Hz')
            plt.legend()
            plt.show()

            # Reproduz o áudio
            display(Audio(self.sinal, rate=taxa_amostragem))
        
        # Cria o ambiente interativo
        interactive_plot = interactive(
            periodico_wrapper,
            amp=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=self.amp),
            duracao=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=self.duracao),
            freq=widgets.IntSlider(min=1, max=440, step=10, value=1)
        )
        display(interactive_plot)

#### Onda senoidal

A **onda senoidal** é um sinal sonoro 

In [143]:
class OndaSenoidal(Onda):
    def __init__(self, amp=1.0, duracao=1.0, freq=440):
        super().__init__(amp, duracao, freq)
    
    def gera_sinal(self, amp=1.0, duracao=1.0, freq=440):
        amostras = int(taxa_amostragem * duracao)
        t = np.linspace(0, duracao, amostras)
        return amp * np.sin(2 * np.pi * freq * t)

In [144]:
onda_senoidal = OndaSenoidal()
onda_senoidal.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=1.0, description='amp', max=1.0, min=0.1), FloatSlider(value=1.0, desc…

#### Onda quadrada

A **onda quadrada** é um sinal sonoro 

In [145]:
class OndaQuadrada(Onda):
    def __init__(self, amp=1.0, duracao=1.0, freq=440):
        super().__init__(amp=amp, duracao=duracao, freq=freq)

    def gera_sinal(self, amp=1.0, duracao=1.0, freq=1):
        amostras = int(taxa_amostragem * duracao)
        t = np.linspace(0, duracao, amostras)
        return amp * signal.square(2 * np.pi * freq * t)

In [146]:
onda_quadrada = OndaQuadrada()
onda_quadrada.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=1.0, description='amp', max=1.0, min=0.1), FloatSlider(value=1.0, desc…

#### Onda dente-de-serra

A **onda dente-de-serra** é um sinal sonoro

In [147]:
class OndaDenteDeSerra(Onda):
    def __init__(self, amp=1.0, duracao=1.0, freq=440):
        super().__init__(amp=amp, duracao=duracao, freq=freq)

    def gera_sinal(self, amp=1.0, duracao=1.0, freq=1):
        amostras = int(taxa_amostragem * duracao)
        t = np.linspace(0, duracao, amostras)
        return amp * signal.sawtooth(2 * np.pi * freq * t)

In [148]:
onda_dente_de_serra = OndaDenteDeSerra()
onda_dente_de_serra.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=1.0, description='amp', max=1.0, min=0.1), FloatSlider(value=1.0, desc…

#### Onda triangular

A **onda triangular** é um sinal sonoro 

In [149]:
class OndaTriangular(Onda):
    def __init__(self, amp=1.0, duracao=1.0, freq=440):
        super().__init__(amp=amp, duracao=duracao, freq=freq)

    def gera_sinal(self, amp=1.0, duracao=1.0, freq=1):
        amostras = int(taxa_amostragem * duracao)
        t = np.linspace(0, duracao, amostras)
        return amp * signal.sawtooth(2 * np.pi * freq * t, width=0.5)

In [150]:
onda_triangular = OndaTriangular()
onda_triangular.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=1.0, description='amp', max=1.0, min=0.1), FloatSlider(value=1.0, desc…

## ADSR

O som de um instrumento musical não é composto apenas pelo seu caráter harmônico, mas também pela sua **variação dinâmica** ao longo do tempo.

---

*O quanto demora para o som atingir o volume máximo?*

*O quanto demora para o som atingir o volume mínimo?*

*O quanto o som mantém o volume máximo?*

*O quanto demora para o som atingir o volume zero?*

---

Essas perguntas são respondidas pelo conceito de **envelope sonoro**.

Um **envelope sonoro** é uma função que descreve a variação da amplitude de um sinal ao longo do tempo.

Um dos envelopes mais usados no mundo da síntese sonora é o **ADSR**, um acrônimo para **Attack**, **Decay**, **Sustain** e **Release**. Aplicado a um sinal sonoro, o ADSR colabora para a percepção um som como parte de um "instrumento musical".

Vamos ouvir alguns exemplos de ADSR em ação.

> *Hora de abrir o Ableton Live...*

Vamos estudar cada uma das etapas do ADSR:

1. **Attack**: é o tempo que o sinal leva para atingir o seu valor máximo. O ataque é o início do som.
2. **Decay**: é o tempo que o sinal leva para atingir o valor de sustentação.
3. **Sustain**: é o valor que o sinal mantém enquanto a tecla do instrumento é pressionada.
4. **Release**: é o tempo que o sinal leva para atingir o valor zero após a tecla do instrumento ser solta.

Podemos modelar cada uma dessas etapas com funções matemáticas. Para fins didáticos, vamos simplificar essas funções para funções lineares.

In [ ]:
class ADSR:
    def __init__(self, a=0.1, d=0.1, s=0.7, r=0.2, sinal=onda_senoidal):
        self.attack = a
        self.decay = d
        self.sustain = s
        self.release = r
        self.sinal = sinal
    
    def aplica_envelope(self, sinal):
        # Cria uma cópia do sinal para não modificar o original
        sinal_adsr = np.copy(sinal)
        
        # Obtém o número de amostras total
        num_amostras = len(sinal)
        
        # Converte tempos para número de amostras
        a_samples = int(self.attack * taxa_amostragem)
        d_samples = int(self.decay * taxa_amostragem)
        r_samples = int(self.release * taxa_amostragem)
        
        # Calcula o ponto de sustain
        sustain_start = a_samples + d_samples
        sustain_end = num_amostras - r_samples
        
        # Envelope ADSR
        envelope = np.ones(num_amostras)
        
        # Ataque (crescimento linear de 0 a 1)
        if a_samples > 0:
            envelope[:a_samples] = np.linspace(0, 1, a_samples)
        
        # Decay (decrescimento linear de 1 a s)
        if d_samples > 0:
            envelope[a_samples:sustain_start] = np.linspace(1, self.sustain, d_samples)
        
        # Sustain (valor constante s)
        envelope[sustain_start:sustain_end] = self.sustain
        
        # Release (decrescimento linear de s a 0)
        if r_samples > 0:
            envelope[sustain_end:] = np.linspace(self.sustain, 0, r_samples)
        
        # Aplica o envelope ao sinal
        sinal_adsr = sinal_adsr * envelope
        
        return sinal_adsr
    
    def cria_ambiente_interativo(self):
        # Cria uma função wrapper para ser chamada pelo widget interativo
        def adsr_wrapper(attack, decay, sustain, release, sinal):
            # Atualiza os parâmetros
            self.attack = attack
            self.decay = decay
            self.sustain = sustain
            self.release = release
            self.sinal = sinal 
            
            # Gera um sinal
            sinal = self.sinal.gera_sinal(freq=440, duracao=1.0)
            
            # Aplica o envelope ADSR
            sinal_adsr = self.aplica_envelope(sinal)
            
            # Gera o gráfico
            plt.figure(figsize=(18, 6))
            plt.plot(sinal, 'b-', label='Sinal Original')
            plt.plot(sinal_adsr, 'r-', label='Sinal ADSR')
            plt.grid(True)
            plt.xlabel('Amostras')
            plt.ylabel('Amplitude')
            plt.ylim(-1.1, 1.1)
            plt.title('Sinal com Envelope ADSR')
            plt.legend()
            plt.show()
            
            # Reproduz o sinal
            display(Audio(sinal_adsr, rate=taxa_amostragem))
        
        # Cria o ambiente interativo
        interactive_plot = interactive(
            adsr_wrapper,
            attack=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.attack),
            decay=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.decay),
            sustain=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.sustain),
            release=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=self.release),
            sinal=widgets.Dropdown(
                options={
                    'Ruído Branco': ruido_branco,
                    'Ruído Vermelho': ruido_vermelho,
                    'Senoidal': onda_senoidal,
                    'Quadrada': onda_quadrada,
                    'Dente de Serra': onda_dente_de_serra,
                    'Triangular': onda_triangular
                },
                value=self.sinal,
                description='Sinal:'
            )
        )
        display(interactive_plot)

In [152]:
adsr = ADSR()
adsr.cria_ambiente_interativo()

interactive(children=(FloatSlider(value=0.1, description='attack', max=1.0, step=0.01), FloatSlider(value=0.1,…

## Modulação AM